<a href="https://colab.research.google.com/github/FreddyMS/COP4777-Data-Mining-Course-Fall-2022/blob/main/CrateandBarrelxgboostParameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#This program took multiple tries, endless tries to get a result of 0.45.
#This program was completed in one day, possibly with more time I could drop under .40
#This is my career choice so it was fascinating to see differ outcome with differ algorithms
#This notebook is the most refined and commented after multiple test runs on other notebooks.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')
!nvidia-smi
#unzippig the zip file
!unzip otto-group-product-classification-challenge.zip
#Looking for files
!ls
#setting up the train, test data and cvs sample and indexing them
otto_train= pd.read_csv('train.csv',index_col=0)
otto_test= pd.read_csv('test.csv',index_col=0)
otto_sample= pd.read_csv('sampleSubmission.csv',index_col=0)

Fri Dec  9 06:25:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    50W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from xgboost.compat import LabelEncoder
#Setting the feature and dropping the label
bFeatures = otto_train.drop(['target'], axis=1)
#calling the dummies function
ottoSparse=pd.get_dummies(bFeatures)
#Setting my label
ottoLabels = otto_train['target']

otto_train_features=ottoSparse
otto_train_labels =ottoLabels

In [8]:
import xgboost
from xgboost import XGBClassifier
#Attempting to get the best paramters for the xgboost, will take around couple of minutes to load with gpu on
params = { "n_estimators": 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor' }
model = XGBClassifier(**params)
param_grid = [
    {
      "n_estimators": [ 50, 100, 150, 200], 
    # 'tree_method':'gpu_hist',
    # 'predictor':'gpu_predictor',
     'max_depth': [2, 4, 6, 8]
    }
   
  ]

#This program will fail out gpu when it does the fitting 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
param_comb = 5
folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(otto_train_features, otto_train_labels), verbose=3)
%time


#"Fitting 5 folds for each of 5 candidates, totalling 25 fits" 
grid_result = random_search.fit(otto_train_features, otto_train_labels)

#model.fit(otto_train_features, otto_train_labels)
from sklearn.metrics import accuracy_score
random_search.best_params_
#probabilities with grid result fitting
predictions_proba = grid_result.predict_proba(otto_test)
print(len(predictions_proba))
print(predictions_proba)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
Fitting 5 folds for each of 5 candidates, totalling 25 fits
144368
[[2.22088667e-04 1.88538685e-01 1.71397537e-01 ... 5.14055416e-03
  3.52256960e-04 8.12283615e-05]
 [5.07975405e-04 5.37672220e-03 1.96922128e-03 ... 1.92255911e-03
  3.90947104e-01 1.54385483e-03]
 [4.74584085e-05 1.27469975e-05 1.22281226e-05 ... 3.96024225e-05
  1.14223687e-03 2.36372671e-05]
 ...
 [1.08209916e-03 4.74541128e-01 4.18124050e-01 ... 1.78979384e-03
  2.85546150e-04 9.83793434e-05]
 [2.13989348e-04 2.32234940e-01 2.63943784e-02 ... 3.39004560e-04
  2.78561965e-05 3.07847840e-05]
 [2.20021146e-04 5.29035509e-01 3.84304821e-01 ... 5.18069640e-02
  2.06476092e-04 2.70492485e-04]]


In [9]:
#Reusing the sample cvs file and setting the index column
sampleIndex=otto_sample.index
sampleIndex= sampleIndex.values

#Removing item vales from the sample cvs file
sampleCol =otto_sample.columns

# Enter the probability for each class, index and set columns 
new_dataframe= pd.DataFrame(predictions_proba, index=sampleIndex, columns= sampleCol)
new_dataframe

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
1,0.000222,0.188539,0.171398,0.633921,2.440723e-05,0.000322,0.005141,0.000352,0.000081
2,0.000508,0.005377,0.001969,0.000835,2.364268e-04,0.596661,0.001923,0.390947,0.001544
3,0.000047,0.000013,0.000012,0.000007,4.314574e-07,0.998715,0.000040,0.001142,0.000024
4,0.000336,0.669001,0.313614,0.015528,3.687116e-05,0.000214,0.000175,0.000252,0.000843
5,0.035509,0.000119,0.000365,0.000074,2.964621e-05,0.001907,0.001903,0.012539,0.947554
...,...,...,...,...,...,...,...,...,...
144364,0.612239,0.007835,0.010300,0.005325,1.484149e-04,0.261773,0.028158,0.020205,0.054016
144365,0.000456,0.320548,0.556108,0.084861,9.297799e-05,0.001151,0.036318,0.000213,0.000251
144366,0.001082,0.474541,0.418124,0.103473,6.626909e-05,0.000540,0.001790,0.000286,0.000098
144367,0.000214,0.232235,0.026394,0.740703,2.213082e-05,0.000034,0.000339,0.000028,0.000031


In [10]:
#Create the cvs file
new_dataframe.to_csv("OttoGroupProjectxgboostParameters.csv", index_label= 'id')